In [680]:
import lightgbm as lgb
import pandas as pd
import numpy as np

model = lgb.Booster(model_file='lgb_classifier.txt')




In [681]:
company = 'Fidelity500'
df_portfolios = pd.DataFrame(pd.read_csv('C:/Users/thy10/Desktop/AREIX/AREIX Jupyter/df_portfolios_'+company+'.csv', header = 0, index_col=0))
print(df_portfolios.shape)
  
X= df_portfolios.iloc[:,[2,3,4,5]].values
y_pred = model.predict(X)
print(y_pred)
for i in range(len(y_pred)):
    if (y_pred[i] >= 0.4): #0.4
        y_pred[i] = 1
    else:
        y_pred[i] = 0
        
#convert result to list
clist=y_pred.tolist()
#add result to df_portfolios
df_portfolios['lgbm']=clist
#output df_portfolios as csv
df_portfolios.to_csv('df_portfolios_'+company+'(bt).csv')
print("=== bt output as csv file ===")

(959, 9)
[0.27301002 0.25441787 0.40665533 0.40665533 0.34450326 0.40665533
 0.40665533 0.40665533 0.40665533 0.40665533 0.21723538 0.25441787
 0.25441787 0.40665533 0.40665533 0.25441787 0.25441787 0.76379942
 0.76379942 0.40665533 0.76379942 0.40665533 0.34450326 0.25441787
 0.76379942 0.76379942 0.40665533 0.40665533 0.34450326 0.27301002
 0.41816387 0.34450326 0.27301002 0.27301002 0.34450326 0.27301002
 0.34450326 0.6798081  0.6798081  0.27301002 0.34450326 0.34450326
 0.34450326 0.6798081  0.76379942 0.76379942 0.34450326 0.34450326
 0.6798081  0.34450326 0.27301002 0.27301002 0.34450326 0.27301002
 0.27301002 0.76379942 0.41816387 0.41816387 0.40665533 0.40665533
 0.41816387 0.33929671 0.33929671 0.31816602 0.41816387 0.34450326
 0.6798081  0.33929671 0.33929671 0.33929671 0.34450326 0.27301002
 0.27301002 0.41816387 0.41816387 0.6798081  0.6798081  0.6798081
 0.41816387 0.34450326 0.27301002 0.27301002 0.34450326 0.34450326
 0.6798081  0.40665533 0.76379942 0.41816387 0.3445032

In [682]:
print("--- Number of flags ---")
print(df_portfolios['lgbm'].value_counts())

--- Number of flags ---
1.0    481
0.0    478
Name: lgbm, dtype: int64


## Back-testing

In [683]:
from datetime import date
def get_start_date():
    if (company[:-3]=='Allianz'):
        start_date = date(2001,2,1)
        
    else:
        start_date = date(2000,12,1)
        
    return start_date

In [684]:

'''
1.Group by month 
2.Check if there is any "1" at the column 'lgbm'
2.1 find one and keep the "1" with largest port_return
2.2 if there is no one, check the check the column 'port_return' to find the largest one
--> result, get the best weighting for the current month
'''

df_one = df_portfolios[df_portfolios.lgbm==1]
df_zero=df_portfolios[df_portfolios.lgbm==0]

df_best_one = df_one.groupby(level=0,group_keys=False).apply(lambda x: x.loc[x['port_return']==x['port_return'].max()])
df_best_zero = df_zero.groupby(level=0,group_keys=False).apply(lambda x: x.loc[x['port_return']==x['port_return'].max()])

#df_best is the df that contains the best one in the month
df_best = df_best_one.append([df_best_zero], sort=True)
df_best = df_best.groupby(level=0,group_keys=False).apply(lambda x: x.loc[x['lgbm']==x['lgbm'].max()])
df_best = df_best.groupby(level=0,group_keys=False).apply(lambda x: x.loc[x['next_month_port_return']==x['next_month_port_return'].max()])
df_best = df_best.groupby(level=0,group_keys=False).tail(1)

df_best.index=pd.to_datetime(df_best.index, format="%Y-%m").to_period('M')
df_best=df_best.sort_index()
df_best.to_csv('df_best_'+company+'(bt).csv')
print(df_best.head(10))
print(len(df_best))

                               A_name  A_weight  \
Date                                              
2000-12  Fidelity Capital Stable Fund  0.382010   
2001-01  Fidelity Capital Stable Fund  0.353678   
2001-02  Fidelity Capital Stable Fund  0.363851   
2001-03  Fidelity Capital Stable Fund  0.361732   
2001-04  Fidelity Capital Stable Fund  0.354541   
2001-05  Fidelity Capital Stable Fund  0.362578   
2001-06  Fidelity Capital Stable Fund  0.369670   
2001-07  Fidelity Capital Stable Fund  0.351466   
2001-08  Fidelity Capital Stable Fund  0.402340   
2001-09  Fidelity Capital Stable Fund  0.375760   

                                 B_name  B_weight  flag  lgbm  \
Date                                                            
2000-12  Fidelity Hong Kong Equity Fund  0.617990     0   0.0   
2001-01  Fidelity Hong Kong Equity Fund  0.646322     1   0.0   
2001-02  Fidelity MPF Conservative Fund  0.636149     1   1.0   
2001-03  Fidelity MPF Conservative Fund  0.638268     0   1.0 

In [685]:
from dateutil.relativedelta import relativedelta
start_date=get_start_date()
start_date = start_date + relativedelta(months=+1)

ind = pd.date_range(start_date, periods=len(df_best.index),freq='M')
print(ind)

DatetimeIndex(['2001-01-31', '2001-02-28', '2001-03-31', '2001-04-30',
               '2001-05-31', '2001-06-30', '2001-07-31', '2001-08-31',
               '2001-09-30', '2001-10-31',
               ...
               '2020-01-31', '2020-02-29', '2020-03-31', '2020-04-30',
               '2020-05-31', '2020-06-30', '2020-07-31', '2020-08-31',
               '2020-09-30', '2020-10-31'],
              dtype='datetime64[ns]', length=238, freq='M')


In [686]:
#create new df
cols=['A_name','B_name','A_weight','B_weight','lgbm']
df_bt = df_best.loc[:,cols]
df_bt = df_bt.reset_index(drop=True)
df_bt.set_index(ind, inplace=True)
df_bt.index=pd.to_datetime(df_bt.index, format="%Y-%m").to_period('M')
print(df_bt)

                                    A_name                           B_name  \
2001-01       Fidelity Capital Stable Fund   Fidelity Hong Kong Equity Fund   
2001-02       Fidelity Capital Stable Fund   Fidelity Hong Kong Equity Fund   
2001-03       Fidelity Capital Stable Fund   Fidelity MPF Conservative Fund   
2001-04       Fidelity Capital Stable Fund   Fidelity MPF Conservative Fund   
2001-05       Fidelity Capital Stable Fund             Fidelity Growth Fund   
...                                    ...                              ...   
2020-06        Fidelity Global Equity Fund           Fidelity Balanced Fund   
2020-07  Fidelity Asia Pacific Equity Fund   Fidelity Hong Kong Equity Fund   
2020-08  Fidelity Asia Pacific Equity Fund     Fidelity Hong Kong Bond Fund   
2020-09        Fidelity Global Equity Fund  Fidelity Hong Kong Tracker Fund   
2020-10             Fidelity RMB Bond Fund     Fidelity Hong Kong Bond Fund   

         A_weight  B_weight  lgbm  
2001-01  0.3820

In [687]:
def calculate_port_vol(df_daily,  month_index,A_name, B_name, A_weight, B_weight):
    #df_daily, df_merge_monthly, month_index, A_name, Available_B[list_num][1], A_weight, B_weight
    #month_index=month_index.strftime('%Y-%m-%d')
    port_list =[]
    weights = np.array([A_weight, B_weight])

    port_list.append(A_name)
    port_list.append(B_name)
    
    df_vol = df_daily.loc[:month_index,port_list]
    
    df_vol.replace('', np.nan, inplace=True)
    df_vol = df_vol.dropna(how='any', axis=0)
    
    if(len(df_vol)==1):
        return ''
    
    cov_matrix_annual = df_vol.cov() * 252

    port_variance = np.dot(weights.T, np.dot(cov_matrix_annual, weights))
    port_volatility = np.sqrt(port_variance)

    return port_volatility 

In [688]:
companyidx=company[:-3]
df = pd.DataFrame(pd.read_csv("C:/Users/thy10/Desktop/AREIX/AREIX Jupyter/" + companyidx+".csv", index_col='Valuation Date', parse_dates=True))
df_daily=df.pct_change()
df_monthly=pd.DataFrame(pd.read_csv("C:/Users/thy10/Desktop/AREIX/AREIX Jupyter/" + companyidx+"_df_monthly.csv", index_col='Valuation Date', parse_dates=True))
df_monthly.index=pd.to_datetime(df_monthly.index, format="%Y-%m").to_period('M')
results=[]
#port_return=[]

A_name=df_bt.iloc[0]['A_name']
B_name=df_bt.iloc[0]['B_name']
A_weight=df_bt.iloc[0]['A_weight']
B_weight=df_bt.iloc[0]['B_weight']

for index, row in df_bt.iterrows():
    #print(df_monthly.loc[index,df_monthly['A_name']==A_name])
  
    
    if(row['lgbm']==1):  #if it is equal to zero, then you can use this portfolio for this month
        A_name=row['A_name']
        B_name=row['B_name']
        A_weight=row['A_weight']
        B_weight=row['B_weight']
        
    port_return=df_monthly.loc[index,A_name]*A_weight+df_monthly.loc[index,B_name]*B_weight    
    this_month_index=index.strftime('%Y-%m-%d')
    port_vol = calculate_port_vol(df_daily,this_month_index,A_name, B_name, A_weight, B_weight)
    
    results.append([index,A_name,B_name,A_weight,B_weight,port_return,port_vol])
 
df_bt_results = pd.DataFrame(results,columns = ["Date","A_name","B_name","A_weight","B_weight","port_return","port_vol"])   
df_bt_results.set_index("Date", inplace = True) 

print(df_bt_results)
df_bt_results.to_csv('df_bt_results_'+company+'.csv')

                                    A_name                           B_name  \
Date                                                                          
2001-01       Fidelity Capital Stable Fund   Fidelity Hong Kong Equity Fund   
2001-02       Fidelity Capital Stable Fund   Fidelity Hong Kong Equity Fund   
2001-03       Fidelity Capital Stable Fund   Fidelity MPF Conservative Fund   
2001-04       Fidelity Capital Stable Fund   Fidelity MPF Conservative Fund   
2001-05       Fidelity Capital Stable Fund   Fidelity MPF Conservative Fund   
...                                    ...                              ...   
2020-06        Fidelity Global Equity Fund           Fidelity Balanced Fund   
2020-07        Fidelity Global Equity Fund           Fidelity Balanced Fund   
2020-08  Fidelity Asia Pacific Equity Fund     Fidelity Hong Kong Bond Fund   
2020-09        Fidelity Global Equity Fund  Fidelity Hong Kong Tracker Fund   
2020-10             Fidelity RMB Bond Fund     Fidel